In [13]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import cv2
import numpy as np
import torchvision.transforms as transforms
import tqdm
from torch.utils.data import Dataset
from tqdm import tqdm
import pandas as pd
import matplotlib.pyplot as plt
import os

In [14]:
import sys
sys.path.append("..")

In [15]:
class ImageDataset(Dataset):
    def __init__(self, csv, train, test):
        self.csv = csv
        self.train = train
        self.test = test
        self.all_image_names = self.csv[:]["filename"]
        self.all_labels = np.array(self.csv.drop(["filename", "ind"], axis=1))
        self.train_ratio = int(0.85 * len(self.csv))
        self.valid_ratio = len(self.csv) - self.train_ratio
        # set the training data images and labels
        if self.train == True:
            print(f"Number of training images: {self.train_ratio}")
            self.image_names = list(self.all_image_names[: self.train_ratio])
            self.labels = list(self.all_labels[: self.train_ratio])
            # define the training transforms
            self.transform = transforms.Compose(
                [
                    transforms.ToTensor(),
                ]
            )
        # set the validation data images and labels
        elif self.train == False and self.test == False:
            print(f"Number of validation images: {self.valid_ratio}")
            self.image_names = list(self.all_image_names[-self.valid_ratio : -10])
            self.labels = list(self.all_labels[-self.valid_ratio :])
            # define the validation transforms
            self.transform = transforms.Compose(
                [
                    transforms.ToTensor(),
                ]
            )
        # set the test data images and labels, only last 10 images
        # this, we will use in a separate inference script
        elif self.test == True and self.train == False:
            self.image_names = list(self.all_image_names[-10:])
            self.labels = list(self.all_labels[-10:])
            # define the test transforms
            self.transform = transforms.Compose(
                [
#                     transforms.ToPILImage(),
#                     transforms.Resize((400, 400)),
                    transforms.ToTensor(),
                ]
            )

    def __len__(self):
        return len(self.image_names)

    def __getitem__(self, index):
        image = cv2.imread(f"../data/raw_images/{self.image_names[index]}")
        # apply image transforms
        image = self.transform(image)
        targets = self.labels[index]

        return {
            "image": torch.tensor(image, dtype=torch.float32),
            "label": torch.tensor(targets, dtype=torch.float32),
        }

In [16]:
class LeNet5_convnet(nn.Module):
    def __init__(self):

        super(LeNet5_convnet, self).__init__()

        # CL1: 3 x 256 x 144 --> 50 x 256 x 144
        self.conv1 = nn.Conv2d(3, 50, kernel_size=3, padding='same')

        # MP1: 50 x 256 x 144 --> 50 x 128 x 72
        self.pool1 = nn.MaxPool2d(2, 2)

        # CL2: 50 x 128 x 72 --> 100 x 128 x 72
        self.conv2 = nn.Conv2d(50, 100, kernel_size=3, padding='same')

        # MP2: 100 x 128 x 72 --> 100 x 64 x 36
        self.pool2 = nn.MaxPool2d(2, 2)

        # LL1:   100 x 64 x 36 = 230400 -->  100
        self.linear1 = nn.Linear(230400, 100)

        # LL2:   100  -->  6
        self.linear2 = nn.Linear(100, 6)

    def forward(self, x):

        # CL1: 3 x 256 x 144 --> 50 x 256 x 144
        x = self.conv1(x)
        x = F.relu(x)

        # MP1: 50 x 256 x 144 --> 50 x 128 x 72
        x = self.pool1(x)

        # CL2: 50 x 128 x 72 --> 100 x 128 x 72
        x = self.conv2(x)
        x = F.relu(x)

        # MP2: 100 x 128 x 72 --> 100 x 64 x 36
        x = self.pool2(x)

        # LL1:   100 x 64 x 36 = 230400 -->  100
        x = x.view(-1, 230400)
        x = self.linear1(x)
        x = F.relu(x)

        # LL2:   100  -->  6
        x = self.linear2(x)

        return x


In [17]:
# training function
def train(model, dataloader, optimizer, criterion, train_data, device):
    print("Training")
    model.train()
    counter = 0
    train_running_loss = 0.0
    for i, data in tqdm(
        enumerate(dataloader), total=int(len(train_data) / dataloader.batch_size)
    ):
        counter += 1
        data, target = data["image"].to(device), data["label"].to(device)
        optimizer.zero_grad()
        outputs = model(data)
        # apply sigmoid activation to get all the outputs between 0 and 1
        outputs = torch.sigmoid(outputs)
        loss = criterion(outputs, target)
        train_running_loss += loss.item()
        # backpropagation
        loss.backward()
        # update optimizer parameters
        optimizer.step()

    train_loss = train_running_loss / counter
    return train_loss

In [18]:
def validate(model, dataloader, criterion, val_data, device):
    print("Validating")
    model.eval()
    counter = 0
    val_running_loss = 0.0
    with torch.no_grad():
        for i, data in tqdm(
            enumerate(dataloader), total=int(len(val_data) / dataloader.batch_size)
        ):
            counter += 1
            data, target = data["image"].to(device), data["label"].to(device)
            outputs = model(data)
            # apply sigmoid activation to get all the outputs between 0 and 1
            outputs = torch.sigmoid(outputs)
            loss = criterion(outputs, target)
            val_running_loss += loss.item()

        val_loss = val_running_loss / counter
        return val_loss

In [19]:
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib
from torch.utils.data import DataLoader

matplotlib.style.use("ggplot")
# initialize the computation device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [20]:
model = LeNet5_convnet()
net = model.to(device)

lr = 0.0001
epochs = 10
batch_size = 32
optimizer = optim.Adam(model.parameters(), lr=lr)
criterion = nn.BCELoss()

In [21]:
train_csv = pd.read_csv("../data/processed/train.csv")
# train dataset
train_data = ImageDataset(train_csv[:1000], train=True, test=False)
# validation dataset
valid_data = ImageDataset(train_csv[:1000], train=False, test=False)
# train data loader
train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
# validation data loader
valid_loader = DataLoader(valid_data, batch_size=batch_size, shuffle=False)



Number of training images: 849
Number of validation images: 150


In [22]:
# mean = train_data.mean()
# std = train_data.std()
# mean=mean.to(device)
# std=std.to(device)

In [23]:
train_loss = []
valid_loss = []
for epoch in range(epochs):
    print(f"Epoch {epoch+1} of {epochs}")
    train_epoch_loss = train(
        model, train_loader, optimizer, criterion, train_data, device
    )
    valid_epoch_loss = validate(model, valid_loader, criterion, valid_data, device)
    train_loss.append(train_epoch_loss)
    valid_loss.append(valid_epoch_loss)
    print(f"Train Loss: {train_epoch_loss:.4f}")
    print(f"Val Loss: {valid_epoch_loss:.4f}")
    

Epoch 1 of 10
Training


  0%|                                                                                           | 0/26 [00:00<?, ?it/s]C:\Users\Rishabh\anaconda3\envs\deeplearn_course\lib\site-packages\ipykernel_launcher.py:62: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
C:\Users\Rishabh\anaconda3\envs\deeplearn_course\lib\site-packages\torch\nn\functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  ..\c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)
27it [00:03,  8.47it/s]                                                                                                


Validating


5it [00:00, 20.98it/s]                                                                                                 


Train Loss: 0.4402
Val Loss: 0.4267
Epoch 2 of 10
Training


...

Train Loss: 0.0741
Val Loss: 0.6652
Epoch 10 of 10
Training


27it [00:01, 17.47it/s]                                                                                                


Validating


5it [00:00, 22.38it/s]                                                                                                 

Train Loss: 0.0623
Val Loss: 0.7388
